<a href="https://colab.research.google.com/github/rajiv6741/ML_Assignment_1/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [ ]:
dataset = pd.read_csv('/content/data.csv')
labels = pd.read_csv('/content/label.csv',names=['label'],header=None)

In [ ]:
dataset.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_values, test_values = train_test_split( dataset, test_size=0.08, random_state=50)
train_label_val, test_label_val = train_test_split( labels, test_size=0.08, random_state=50)

In [ ]:
def calculate_SSE(centroid_value_dict, centroid_dict,data):
    sse = 0
    for i in centroid_dict:
        sse_cluster = 0
        for j in centroid_dict[i]:
            dp = list(data.iloc[int(j)])
            for a,b in zip(centroid_value_dict[i],dp):
                sse_cluster += (a-b)**2
        sse+=sse_cluster
    return sse   
    
def Initialize_Centroids(data,K):
    p = data.shape[0]
    centroid_value_dict={}
    for i in range(K):
        r = np.random.randint(0, p-1)
        centroid_value_dict[i] = data.iloc[r]
    return centroid_value_dict

def jaccard_similarity(centroid, dp):
    top = len(list(set(centroid).intersection(dp)))
    bottom = (len(set(centroid)) + len(set(dp))) - top
    return float(top) / bottom

def train_Kmeans(data,K,max_iter=20,mode=1,tol=10):
    centroid_value_dict = Initialize_Centroids(data,K)
    count = 0
    centroid_dict = {}
    flag = False
    while((count<max_iter) and not flag):
            
        for i in list(centroid_value_dict.keys()):
            centroid_dict[i]=[]
        for i in range(data.shape[0]):
            x = data.iloc[i]
            if mode==1 :
                distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                idx = np.argmin(distance_measure)
                centroid_dict[idx].append(i)
            elif mode==2 :
                distance_measure = [jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                idx = np.argmax(distance_measure)
                centroid_dict[idx].append(i)
            elif mode==3 :
                distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                idx = np.argmax(distance_measure)
                centroid_dict[idx].append(i)
                
            prev_centroids=dict(centroid_value_dict)
        for i in centroid_dict:
            if len(centroid_dict[i]):
                dps_centroid = centroid_dict[i]
                centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
        current_tol=-1
        for i in centroid_value_dict:
            prev_centroid_point = prev_centroids[i]
            new_centroid_point = centroid_value_dict[i]
            change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
            current_tol = max(change, current_tol)
                
        print("Iteration ",count,": ",current_tol)
            
        count+=1
        if (current_tol<10):
            flag = True
            break
    return centroid_value_dict,centroid_dict


In [ ]:
def predict_cluster_labels(C, S, labels):
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [ ]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [ ]:
centroids1,clusters1 = train_Kmeans(dataset,10, max_iter=100,mode=1)

Iteration  0 :  24819.69417475728
Iteration  1 :  5042.6196489652075
Iteration  2 :  4435.3252231335755
Iteration  3 :  2795.6647304933076
Iteration  4 :  1825.878066878434
Iteration  5 :  1134.803734356789
Iteration  6 :  1036.4577159076189
Iteration  7 :  767.4419494484011
Iteration  8 :  730.4394982527512
Iteration  9 :  534.3250304315054
Iteration  10 :  423.0901169084555
Iteration  11 :  368.29960108141927
Iteration  12 :  435.96231945796814
Iteration  13 :  361.8884509237397
Iteration  14 :  336.01974568763336
Iteration  15 :  327.56797382252915
Iteration  16 :  291.51808743530216
Iteration  17 :  345.40826083146607
Iteration  18 :  265.5127219391694
Iteration  19 :  299.6316699191525
Iteration  20 :  309.5673483003214
Iteration  21 :  263.74919594057087
Iteration  22 :  260.6074215102372
Iteration  23 :  208.10616865614168
Iteration  24 :  248.48804346135358
Iteration  25 :  228.5518729389941
Iteration  26 :  190.52960458407574
Iteration  27 :  116.54873158687369
Iteration  28 :

In [ ]:
Euclidean_SSE =calculate_SSE(centroids1,clusters1,dataset)

In [ ]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25317253378.89649


In [ ]:
cluster_labels_euc = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels_euc

array([2, 1, 3, 8, 3, 0, 0, 7, 5, 9])

In [ ]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels_euc,test_values,test_label_val)
Accuracy_Euclidean

0.10875

In [ ]:
centroids2,clusters2 =train_Kmeans(dataset,10, max_iter=100,mode=2)

Iteration  0 :  34547.07046979866
Iteration  1 :  17205.099066069357
Iteration  2 :  2713.931123447135
Iteration  3 :  4528.676502277556
Iteration  4 :  1771.817653519338
Iteration  5 :  1408.4203388080743
Iteration  6 :  1346.8869083588856
Iteration  7 :  1118.809631697482
Iteration  8 :  1351.1196324558564
Iteration  9 :  1033.4210197507416
Iteration  10 :  876.4440942718757
Iteration  11 :  359.42736231033916
Iteration  12 :  209.90812802793226
Iteration  13 :  0.0


In [ ]:
Jaccard_SSE =calculate_SSE(centroids2,clusters2,dataset)

In [ ]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [ ]:
cluster_labels_jac = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels_jac

array([1, 5, 7, 5, 8, 2, 1, 8, 3, 0])

In [ ]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels_jac,test_values,test_label_val,mode=2)
Accuracy_Jaccard

0.1025

In [ ]:
centroids3,clusters3 =train_Kmeans(dataset,10, max_iter = 100,mode=3)

Iteration  0 :  25820.127118644064
Iteration  1 :  5629.405815288495
Iteration  2 :  3100.007275006777
Iteration  3 :  2331.8985219589485
Iteration  4 :  2153.2079095376234
Iteration  5 :  1975.4967265560667
Iteration  6 :  1506.959388763212
Iteration  7 :  1149.2052386995026
Iteration  8 :  783.1159990102003
Iteration  9 :  591.8013832816669
Iteration  10 :  633.8797392824565
Iteration  11 :  683.1853800900817
Iteration  12 :  725.4778931887122
Iteration  13 :  631.4091969192161
Iteration  14 :  452.2210315952897
Iteration  15 :  355.48096166513835
Iteration  16 :  305.2193407110684
Iteration  17 :  285.7553962290097
Iteration  18 :  251.00086768822626
Iteration  19 :  252.90339615899234
Iteration  20 :  326.98025076607325
Iteration  21 :  431.103678469832
Iteration  22 :  308.4124965798195
Iteration  23 :  316.71552152802144
Iteration  24 :  277.3828569306512
Iteration  25 :  192.66634137525963
Iteration  26 :  203.92832587708966
Iteration  27 :  137.18323383084564
Iteration  28 :  1

In [ ]:
Cosine_SSE = calculate_SSE(centroids3,clusters3,dataset)

In [ ]:
cluster_labels_cos = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels_cos

array([3, 5, 9, 3, 1, 0, 4, 8, 2, 7])

In [ ]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels_cos,test_values,test_label_val,mode=3)

In [ ]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.10875
Jacard accuracy: 0.1025
Cosine accuracy : 0.10375


In [ ]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25317253378.89649
Jacard SSE: 34361687572.938736
Cosine SSE : 25570031972.50617
